In [2]:
import numpy as np
import os
import glob
from scipy import interpolate
from netCDF4 import Dataset
import pandas as pd
from global_land_mask import globe
from sklearn.externals import joblib 

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)

/home/ajiwiranto/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
#split data
from sklearn.model_selection import train_test_split

#ML_model_random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

#metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix,classification_report

In [4]:
# model_rainarea = joblib.load('model/M_RA.pkl') 
# model_Type = joblib.load('model/M_RT.pkl')
# model_Rw =joblib.load('model/M_Rw.pkl')
# model_Rs = joblib.load('model/M_Rs.pkl')

In [5]:
# #open model
# %run -i '/home/ajiwiranto/Documents/kodingan/TA/df_hmwr-gpm/random_forest/python/implementation/fix_script/RF_model.py'

# Read Himawari Data

In [11]:
#data himawari
data = sorted(glob.glob('C:data/*.nc'))

In [12]:
data

['/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/python/implementation/fix_script_1/data_studikasus/himawari/coba/bm_tir01_201901010000.nc',
 '/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/python/implementation/fix_script_1/data_studikasus/himawari/coba/bm_tir02_201901010000.nc',
 '/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/python/implementation/fix_script_1/data_studikasus/himawari/coba/bm_tir03_201901010000.nc',
 '/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/python/implementation/fix_script_1/data_studikasus/himawari/coba/bm_tir04_201901010000.nc',
 '/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/python/implementation/fix_script_1/data_studikasus/himawari/coba/bm_tir05_201901010000.nc',
 '/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/python/implementation/fix_script_1/data_studikasus/himawari/coba/bm_tir06_201901010000.nc',
 '/home/ajiwiranto/Documents/Tugas_Akhir/kodingan/random_forest/

In [13]:
key=['b13','b14','b15','b16','b7','b8','b9','b10','b11','b12'] 

IR={}
for i in range(0,np.size(data)):
    IR[key[i]]=Dataset(data[i])['bt'][:]
IR['lon']=Dataset(data[0])['lon'][:]
IR['lat']=Dataset(data[0])['lat'][:]


In [14]:
IR['lon'][:], IR['lat'][:]

(masked_array(data=[105.6 , 105.62, 105.64, 105.66, 105.68, 105.7 , 105.72,
                    105.74, 105.76, 105.78, 105.8 , 105.82, 105.84, 105.86,
                    105.88, 105.9 , 105.92, 105.94, 105.96, 105.98, 106.  ,
                    106.02, 106.04, 106.06, 106.08, 106.1 , 106.12, 106.14,
                    106.16, 106.18, 106.2 , 106.22, 106.24, 106.26, 106.28,
                    106.3 , 106.32, 106.34, 106.36, 106.38, 106.4 , 106.42,
                    106.44, 106.46, 106.48, 106.5 , 106.52, 106.54, 106.56,
                    106.58, 106.6 , 106.62, 106.64, 106.66, 106.68, 106.7 ,
                    106.72, 106.74, 106.76, 106.78, 106.8 , 106.82, 106.84,
                    106.86, 106.88, 106.9 , 106.92, 106.94, 106.96, 106.98,
                    107.  , 107.02, 107.04, 107.06, 107.08, 107.1 , 107.12,
                    107.14, 107.16, 107.18, 107.2 , 107.22, 107.24, 107.26,
                    107.28, 107.3 , 107.32, 107.34, 107.36, 107.38, 107.4 ,
            

In [15]:
#domain 
south, north = -7.2, -6.74
west, east =  107.45, 107.9

In [16]:
hor_res=0.05 #ukuran grid
grid_x, grid_y = np.mgrid[west:east:hor_res, south:north:hor_res]

In [17]:
#interpolasi ke resolusi GPM yg di atas (0.05 deg)
x,y=np.meshgrid(IR['lon'],IR['lat'])
points=np.vstack((x.reshape(-1),y.reshape(-1)))
points=points.T
#
zH={}
for i in range(0,np.size(data)):
    zH[key[i]]=interpolate.griddata(points,IR[key[i]].reshape(-1),(grid_x,grid_y),method='nearest')
zH['lon']=grid_x[:,0]
zH['lat']=grid_y[0,:]

In [28]:
points

masked_array(
  data=[[105.6 ,  -8.5 ],
        [105.62,  -8.5 ],
        [105.64,  -8.5 ],
        ...,
        [114.26,  -5.8 ],
        [114.28,  -5.8 ],
        [114.3 ,  -5.8 ]],
  mask=False,
  fill_value=1e+20)

In [27]:
IR['b13'].reshape(-1).shape

(59296,)

In [22]:
#membuat dataFrame untuk seluruh BT, kombinasi BTD, dan CH GPM 

#ini yg himawari
d1 = {}
d1['lat'] = np.repeat(zH['lat'], zH['lon'].size)
d1['lon'] = np.tile(zH['lon'], zH['lat'].size)
# d1['prec']= zG.T.ravel()
#d1['3.9'] = zH['b7'].T.ravel()
d1['6.2'] = zH['b8'].T.ravel()
d1['6.9'] = zH['b9'].T.ravel()
d1['7.3'] = zH['b10'].T.ravel()
d1['8.6'] = zH['b11'].T.ravel()
d1['9.6'] = zH['b12'].T.ravel()
d1['10.4']= zH['b13'].T.ravel()
d1['11.2']= zH['b14'].T.ravel()
d1['12.4']= zH['b15'].T.ravel()
d1['13.3']= zH['b16'].T.ravel()

d1['6.2-6.9'] = d1['6.2']-d1['6.9']
d1['6.2-9.6'] = d1['6.2']-d1['9.6']

d1['7.3-9.6'] = d1['7.3']-d1['9.6']
d1['7.3-10.4'] = d1['7.3']-d1['10.4']
d1['7.3-12.4'] = d1['7.3']-d1['12.4']
d1['7.3-13.3'] = d1['7.3']-d1['13.3']

d1['8.6-10.4'] = d1['8.6']-d1['10.4']
d1['8.6-12.4'] = d1['8.6']-d1['12.4']

d1['10.4-11.2'] = d1['10.4']-d1['11.2']
d1['10.4-12.4'] = d1['10.4']-d1['12.4']

df1 = pd.DataFrame(d1)
# df.to_excel('HIMAWARI_201903250900.xlsx')

# data_model

model_rainarea 
, model_Type 
, model_Rw 
 ,model_Rs

In [137]:
dat = df1[['lon','lat','10.4-11.2', '8.6-10.4','8.6-12.4','10.4-12.4','6.2-6.9',
           '7.3-10.4', '7.3-9.6','6.2-9.6', '7.3-12.4','7.3-13.3']]

In [138]:
dat

,lon,lat,10.4-11.2,8.6-10.4,8.6-12.4,10.4-12.4,6.2-6.9,7.3-10.4,7.3-9.6,6.2-9.6,7.3-12.4,7.3-13.3
0,107.45,-7.20,0.929596,-1.500000,-0.153778,1.346222,-1.457260,-3.009903,-10.130066,-12.196426,-1.663681,-0.952057
1,107.50,-7.20,2.096924,-1.053574,1.479980,2.533554,-0.961578,-1.911011,-10.064651,-12.303391,0.622543,1.307343
2,107.55,-7.20,1.564148,-1.410751,0.859756,2.270508,-3.840759,-4.602432,-0.360962,-5.760422,-2.331924,0.025208
3,107.60,-7.20,1.789368,-0.469772,3.162201,3.631973,-7.028564,-9.221558,0.266220,-11.862686,-5.589584,-0.765930
4,107.65,-7.20,2.338333,-0.610870,4.048416,4.659286,-5.873489,-12.221466,-3.846298,-14.183167,-7.562180,-2.076462
...,...,...,...,...,...,...,...,...,...,...,...,...
95,107.70,-6.75,2.015259,0.249542,4.438599,4.189056,-7.357727,-19.984711,-8.184311,-22.497360,-15.795654,-6.003922
96,107.75,-6.75,1.368042,-1.973755,0.244675,2.218430,-3.062897,-5.457184,-4.397842,-8.911789,-3.238754,-1.420410
97,107.80,-6.75,1.217133,-1.029465,0.040771,1.070236,-3.531082,-5.350510,-1.918152,-7.611221,-4.280273,-0.816849
98,107.85,-6.75,1.562134,-0.637665,1.364761,2.002426,-7.353867,-7.523636,0.780792,-10.611984,-5.521210,-1.035812


In [139]:
dat1 = dat.iloc[:,2:].copy()

# Implementasi Model

In [140]:
#predict_rainarea
r_area = model_rainarea.predict(dat1)

In [141]:
clen = len(r_area)
flag2 = np.array(list(range(clen)))
cnum = flag2[r_area ==0]
rnum = flag2[r_area ==1]

In [142]:
rain = dat1.iloc[rnum,:]

In [143]:
### To predict rain type ###
r_type = model_Type.predict(rain)

In [144]:
# r_type[r_type>=1.8]

In [145]:
rlen = len(r_type)
flag3 = np.array(list(range(rlen)))
snum = flag3[r_type ==0]
hnum = flag3[r_type ==1]

In [146]:
w_rain= rain.iloc[snum,:]
s_rain= rain.iloc[hnum,:]

In [147]:
### To predict weak rain rate ###
weak_rain = model_Rw.predict(w_rain)

In [148]:
##to predict strong rain###
strong_rain = model_Rs.predict(s_rain)

In [149]:
flag3[snum]= weak_rain
flag3[hnum]= strong_rain

In [150]:
flag2[cnum] = 0.0
flag2[rnum] = flag3

In [151]:
flag2

array([ 5,  7,  7,  0,  0,  6,  4, 12,  0,  0,  0,  0,  3,  5,  0,  0,  0,
        6,  8, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17,  0,  0,  4,  9,
        0,  0,  0, 15,  0,  8,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,
        0,  9,  0,  0, 12,  0, 11,  0,  4,  0,  0,  7,  7,  0,  4])

In [152]:
hujan = flag2.copy()

In [153]:
zH['lat'].shape, zH['lon'].shape

((10,), (10,))

In [154]:
hujan1 = hujan.reshape(10,10) 
lat = zH['lat'].copy()
lon = zH['lon'].copy()

# Make_nc_file for the result rainfall by the model calculation

In [155]:
zH['lat'], zH['lon']

(array([-7.2 , -7.15, -7.1 , -7.05, -7.  , -6.95, -6.9 , -6.85, -6.8 ,
        -6.75]),
 array([107.45, 107.5 , 107.55, 107.6 , 107.65, 107.7 , 107.75, 107.8 ,
        107.85, 107.9 ]))

In [157]:
hujan1 = hujan.reshape(10,10) 
lat = zH['lat'].copy()
lon = zH['lon'].copy()

In [158]:
# #open model
# %run -i '/home/ajiwiranto/Documents/kodingan/TA/df_hmwr-gpm/random_forest/python/implementation/buat_nc.py'

In [159]:
import netCDF4 as nc4

f = nc4.Dataset('bismillah3_2003270900_notune.nc','w', format='NETCDF4') #'w' stands for write

In [160]:
# #open model
# %run -i '/home/ajiwiranto/Documents/kodingan/TA/df_hmwr-gpm/random_forest/python/implementation/fix_script/make_nc.py'

In [161]:
# rain_model = f.createGroup'Hujan_model'

In [162]:
f.createDimension('lat', len(lat))
f.createDimension('lon', len(lon))
#rain_model.createDimension('time', None)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 10

In [163]:
latitude = f.createVariable('Latitude', 'f4', 'lat')
longitude = f.createVariable('Longitude', 'f4', 'lon')
  
#levels = tempgrp.createVariable('Levels', 'i4', 'z')
hujan = f.createVariable('hujan', 'f4', ('lat', 'lon'))
#time = rain_model.createVariable('Time', 'i4', 'time')

In [164]:
#masukin data
latitude[:] = lat
longitude[:] = lon #The "[:]" at the end of the variable instance is necessary
#levels[:] = z
hujan[:,:] = hujan1

In [165]:
f['Latitude'][:], f['Longitude'][:], f['hujan'][:]

(masked_array(data=[-7.2 , -7.15, -7.1 , -7.05, -7.  , -6.95, -6.9 , -6.85,
                    -6.8 , -6.75],
              mask=False,
        fill_value=1e+20,
             dtype=float32),
 masked_array(data=[107.45, 107.5 , 107.55, 107.6 , 107.65, 107.7 , 107.75,
                    107.8 , 107.85, 107.9 ],
              mask=False,
        fill_value=1e+20,
             dtype=float32),
 masked_array(
   data=[[ 5.,  7.,  7.,  0.,  0.,  6.,  4., 12.,  0.,  0.],
         [ 0.,  0.,  3.,  5.,  0.,  0.,  0.,  6.,  8., 13.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 17.],
         [ 0.,  0.,  4.,  9.,  0.,  0.,  0., 15.,  0.,  8.],
         [ 0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 4.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0., 12.],
         [ 0., 11.,  0.,  4.,  0.,  0.,  7

In [166]:
f.close()